### Ingest qualifying json files

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [1]:
from pandas import read_csv,read_json,concat
from glob import glob
from datetime import datetime
from lib import configuration
from lib import common_functions

In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [3]:
spark = common_functions.get_spark_session()

In [4]:
results_schema = StructType(fields=[StructField("resultId", IntegerType(), False),
                                    StructField("raceId", IntegerType(), True),
                                    StructField("driverId", IntegerType(), True),
                                    StructField("constructorId", IntegerType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("grid", IntegerType(), True),
                                    StructField("position", IntegerType(), True),
                                    StructField("positionText", StringType(), True),
                                    StructField("positionOrder", IntegerType(), True),
                                    StructField("points", FloatType(), True),
                                    StructField("laps", IntegerType(), True),
                                    StructField("time", StringType(), True),
                                    StructField("milliseconds", IntegerType(), True),
                                    StructField("fastestLap", IntegerType(), True),
                                    StructField("rank", IntegerType(), True),
                                    StructField("fastestLapTime", StringType(), True),
                                    StructField("fastestLapSpeed", FloatType(), True),
                                    StructField("statusId", StringType(), True)])

In [5]:
folder_path = f"{configuration.bronze_folder_path}/results/inc"
all_files = glob(folder_path + "/**/*.json", recursive = True)

dfs = [read_json(file, lines=True) for file in all_files]
results_df = concat(dfs, ignore_index=True)
results_df

,resultId,raceId_join,driverId_join,constructorId_join,number,grid,position,positionText,positionOrder,points,laps,statusId
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished
...,...,...,...,...,...,...,...,...,...,...,...,...
474,202424,Abu Dhabi Grand Prix2024,kevin_magnussen,haas,20,14,16,16,positionOrder,0,57,+1 Lap
475,202424,Abu Dhabi Grand Prix2024,lawson,rb,30,12,17,17,positionOrder,0,55,Engine
476,202424,Abu Dhabi Grand Prix2024,bottas,sauber,77,9,18,R,positionOrder,0,30,Collision damage
477,202424,Abu Dhabi Grand Prix2024,colapinto,williams,43,20,19,R,positionOrder,0,26,Engine


##### Step 2 - Rename columns and add new columns
1. Rename qualifyingId, driverId, constructorId and raceId
1. Add ingestion_date with current timestamp

In [6]:
results_df["data_source"] = configuration.v_data_source
#results_df["ingestion_date"] = common_functions.get_ingestion_date()
results_df

,resultId,raceId_join,driverId_join,constructorId_join,number,grid,position,positionText,positionOrder,points,laps,statusId,data_source
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished,api
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished,api
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished,api
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished,api
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished,api
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,202424,Abu Dhabi Grand Prix2024,kevin_magnussen,haas,20,14,16,16,positionOrder,0,57,+1 Lap,api
475,202424,Abu Dhabi Grand Prix2024,lawson,rb,30,12,17,17,positionOrder,0,55,Engine,api
476,202424,Abu Dhabi Grand Prix2024,bottas,sauber,77,9,18,R,positionOrder,0,30,Collision damage,api
477,202424,Abu Dhabi Grand Prix2024,colapinto,williams,43,20,19,R,positionOrder,0,26,Engine,api


In [7]:
results_final_df = results_df.rename(columns={"resultId":"result_id","positionText":"position_text","positionOrder":"position_order","statusId":"status"})
results_final_df

,result_id,raceId_join,driverId_join,constructorId_join,number,grid,position,position_text,position_order,points,laps,status,data_source
0,202401,Bahrain Grand Prix2024,max_verstappen,red_bull,1,1,1,1,positionOrder,26,57,Finished,api
1,202401,Bahrain Grand Prix2024,perez,red_bull,11,5,2,2,positionOrder,18,57,Finished,api
2,202401,Bahrain Grand Prix2024,sainz,ferrari,55,4,3,3,positionOrder,15,57,Finished,api
3,202401,Bahrain Grand Prix2024,leclerc,ferrari,16,2,4,4,positionOrder,12,57,Finished,api
4,202401,Bahrain Grand Prix2024,russell,mercedes,63,3,5,5,positionOrder,10,57,Finished,api
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,202424,Abu Dhabi Grand Prix2024,kevin_magnussen,haas,20,14,16,16,positionOrder,0,57,+1 Lap,api
475,202424,Abu Dhabi Grand Prix2024,lawson,rb,30,12,17,17,positionOrder,0,55,Engine,api
476,202424,Abu Dhabi Grand Prix2024,bottas,sauber,77,9,18,R,positionOrder,0,30,Collision damage,api
477,202424,Abu Dhabi Grand Prix2024,colapinto,williams,43,20,19,R,positionOrder,0,26,Engine,api


In [8]:
drivers_df = read_csv(f'{configuration.silver_folder_path}/drivers.csv')
drivers_df = drivers_df[["driver_id","driver_ref"]]
drivers_df

,driver_id,driver_ref
0,1,abate
1,2,abecassis
2,3,acheson
3,4,adams
4,5,ader
...,...,...
854,855,zapico
855,856,zhou
856,857,zonta
857,858,zorzi


In [9]:
constructors_df = read_csv(f'{configuration.silver_folder_path}/constructors.csv')
constructors_df = constructors_df[["constructor_id","constructor_ref"]]
constructors_df

,constructor_id,constructor_ref
0,1,adams
1,2,afm
2,3,ags
3,4,alfa
4,5,alphatauri
...,...,...
207,208,watson
208,209,wetteroth
209,210,williams
210,211,wolf


In [10]:
race_circuits_df = read_csv(f'{configuration.silver_folder_path}/race_circuits.csv')
race_circuits_df = race_circuits_df[["race_id","race_id_join","race_date"]]
race_circuits_df

,race_id,race_id_join,race_date
0,81,German Grand Prix1959,1959-08-02
1,420,Australian Grand Prix1985,1985-11-03
2,436,Australian Grand Prix1986,1986-10-26
3,452,Australian Grand Prix1987,1987-11-15
4,468,Australian Grand Prix1988,1988-11-13
...,...,...,...
1120,319,Belgian Grand Prix1979,1979-05-13
1121,333,Belgian Grand Prix1980,1980-05-04
1122,347,Belgian Grand Prix1981,1981-05-17
1123,362,Belgian Grand Prix1982,1982-05-09


In [11]:
results_final_df = results_final_df.set_index("driverId_join").join(drivers_df.set_index("driver_ref"), lsuffix='driver_', how='inner').reset_index(drop=True)
results_final_df = results_final_df.set_index("constructorId_join").join(constructors_df.set_index("constructor_ref"), lsuffix='constructor_', how='inner').reset_index(drop=True)
results_final_df = results_final_df.set_index("raceId_join").join(race_circuits_df.set_index("race_id_join"), lsuffix='race_circuits_', how='inner').reset_index(drop=True)

results_final_df["ingestion_date"] = results_final_df["race_date"]

results_final_df = results_final_df[["result_id","race_id","driver_id","constructor_id","number","grid","position","position_text","position_order","points","laps","status","data_source","ingestion_date"]]
results_final_df

,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
0,202424,1125,305,6,10,5,7,7,positionOrder,6,58,Finished,api,2024-12-08
1,202424,1125,17,14,14,8,9,9,positionOrder,2,58,Finished,api,2024-12-08
2,202424,1125,745,14,18,13,14,14,positionOrder,0,57,+1 Lap,api,2024-12-08
3,202424,1125,456,75,16,19,3,3,positionOrder,15,58,Finished,api,2024-12-08
4,202424,1125,679,75,55,3,2,2,positionOrder,18,58,Finished,api,2024-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,202419,1120,803,168,1,2,3,3,positionOrder,15,56,Finished,api,2024-10-20
465,202419,1120,595,168,11,9,7,7,positionOrder,6,56,Finished,api,2024-10-20
466,202419,1120,98,171,77,16,17,17,positionOrder,0,55,+1 Lap,api,2024-10-20
467,202419,1120,856,171,24,18,19,19,positionOrder,0,55,+1 Lap,api,2024-10-20


##### Get max result_id already in silver and only append records newer than it.

In [16]:
max_result_id = common_functions.get_max_result_id()
max_result_id

The maximum value in the column 'result_id' is: 202424


202424

In [13]:
max_result_id = common_functions.get_max_result_id()
results_final_df = results_final_df[(results_final_df['result_id']>max_result_id)]
results_final_df

The maximum value in the column 'result_id' is: 202423


,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
0,202424,1125,305,6,10,5,7,7,positionOrder,6,58,Finished,api,2024-12-08
1,202424,1125,17,14,14,8,9,9,positionOrder,2,58,Finished,api,2024-12-08
2,202424,1125,745,14,18,13,14,14,positionOrder,0,57,+1 Lap,api,2024-12-08
3,202424,1125,456,75,16,19,3,3,positionOrder,15,58,Finished,api,2024-12-08
4,202424,1125,679,75,55,3,2,2,positionOrder,18,58,Finished,api,2024-12-08
5,202424,1125,385,85,27,7,8,8,positionOrder,4,58,Finished,api,2024-12-08
6,202424,1125,491,85,20,14,16,16,positionOrder,0,57,+1 Lap,api,2024-12-08
7,202424,1125,569,135,4,1,1,1,positionOrder,25,58,Finished,api,2024-12-08
8,202424,1125,604,135,81,2,10,10,positionOrder,1,58,Finished,api,2024-12-08
9,202424,1125,349,140,44,16,4,4,positionOrder,12,58,Finished,api,2024-12-08


##### Write to output to processed container in parquet format

In [14]:
results_final_df.to_csv(f"{configuration.silver_folder_path}/results.csv", mode='a', header=False, index=False)

In [15]:
df_parquet = read_csv(f'{configuration.silver_folder_path}/results.csv')
df_parquet

,result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,status,data_source,ingestion_date
0,202005,1023,317,4,99.0,19,17,17,positionOrder,0.0,51,+1 Lap,api,2020-08-09
1,202005,1023,634,4,7.0,20,15,15,positionOrder,0.0,51,+1 Lap,api,2020-08-09
2,202005,1023,305,5,10.0,7,11,11,positionOrder,0.0,52,Finished,api,2020-08-09
3,202005,1023,436,5,26.0,16,10,10,positionOrder,1.0,52,Finished,api,2020-08-09
4,202005,1023,456,75,16.0,8,4,4,positionOrder,12.0,52,Finished,api,2020-08-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26744,202424,1125,803,168,1.0,4,6,6,positionOrder,8.0,58,Finished,api,2024-12-08
26745,202424,1125,595,168,11.0,10,20,R,positionOrder,0.0,0,Collision,api,2024-12-08
26746,202424,1125,98,171,77.0,9,18,R,positionOrder,0.0,30,Collision damage,api,2024-12-08
26747,202424,1125,856,171,24.0,15,13,13,positionOrder,0.0,57,+1 Lap,api,2024-12-08
